In [70]:
!pip install arxiv

In [71]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [72]:
api = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max= 200)
wiki = WikipediaQueryRun(api_wrapper=api)

In [73]:
wiki.name

'wikipedia'

In [74]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [75]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [76]:
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

In [77]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

llm = OllamaEmbeddings(model="gemma2:2b")
db = FAISS.from_documents(documents[:10], llm)

In [ ]:
retriever = db.as_retriever()
retriever

In [32]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool = (retriever,"langchain-search",
                        "Search for information about LangSmith. For any questions about LangSmith, you must use this tool")

In [ ]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arxiv = ArxivQueryRun(api_wrapper= arxiv_wrapper)
arxiv.name

In [37]:
tools = [wiki, arxiv, retriever_tool]

In [ ]:
tools

In [39]:
from dotenv import load_dotenv
from langchain_community.llms import Ollama

load_dotenv()
llms = Ollama(model = "gemma2:2b")

In [ ]:
# from langchain import hub

# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

from langchain.prompts import (
    SystemMessagePromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    PromptTemplate,
)

from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=['input'], template='{input}')
        ),
        MessagesPlaceholder(variable_name='agent_scratchpad'),
    ]
)

In [ ]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llms, tools, chat_prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_exec = AgentExecutor(agent= agent ,tools= tools, verbose= True)
agent_exec

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})